In [1]:
import pandas as pd
import networkx as nx
import json
from fiona.crs import CRS

from commons import *

In [2]:
graph = load_graph_from_feed()
nodes_df, edges_df = graph_to_gdfs(graph)

In [3]:
id_name_mapping = nodes_df['name'].to_dict()

edges_df['name'] = edges_df.index.to_frame()[['source', 'target']].apply(
    lambda x: str(id_name_mapping[x[0]]).replace(" ", "_") + "<-TO->" + str(id_name_mapping[x[1]]).replace(" ", "_"), axis=1, raw=True)
edges_df['key'] = 0
attribute_to_add = edges_df.set_index('key', append=True)['name'].to_dict()

nx.set_edge_attributes(graph, attribute_to_add, 'name')

In [4]:
graph_stats, new_nodes_df = evaluate_graph(graph, nodes_df)
graph_stats.to_csv(os.path.join(RESULTS_PATH, "graph_stats.csv"))

In [8]:
full_graph = graph.copy()
cols = [col for col in new_nodes_df.columns if col not in nodes_df.columns]

for col in cols:
    nx.set_node_attributes(full_graph, new_nodes_df[col].to_dict(), col)

In [11]:
nx.write_gml(graph, os.path.join(RESULTS_PATH, "graph.gml"), stringizer=custom_stringizer)
nx.write_gml(full_graph, os.path.join(RESULTS_PATH, "full_graph.gml"), stringizer=custom_stringizer)